# Создание простой RAG системы

Я реализую простую RAG систему как чат с электронным психологом. То есть за данные берем известные книги по психологии

## Шаг 1: Препроцессинг данных и создание эмбедингов
План:
1. Импорт pdf файлов
2. Препроцессинг текста для эмбедингов (разделение на чанки)
3. Эмбендинг чанков текста с помощью эмбендинговой модели
4. Сохранение эмбендингов в файл

## 1. Импорт pdf файлов

In [1]:
import os
import requests

pdf_path = 'psichology books/'
books = ['Allana-Piza-YAzyk-telodvizhenij', 'Eric-Berne-games-that-people-play', 'Goleman-D.-Emotional-Intelligence.-Why-it-may-be-more-important-than-IQ', 'Gurina-Koshenova-Rabota-psikhologa-s_detskoi', 'Psihology-Aykido', 'Robert-Chaldyny_Psyhologyya-vliyaniya_Kak-nauchitsya-ubezhdat-y-dobivatsya-uspeha']

for book in books:
    if os.path.exists(pdf_path + book + '.pdf'):
        print(f'[INFO] File exists. Skipping {book}...')
    else:
        print(f'[INFO] {book} is not found')



[INFO] File exists. Skipping Allana-Piza-YAzyk-telodvizhenij...
[INFO] File exists. Skipping Eric-Berne-games-that-people-play...
[INFO] File exists. Skipping Goleman-D.-Emotional-Intelligence.-Why-it-may-be-more-important-than-IQ...
[INFO] File exists. Skipping Gurina-Koshenova-Rabota-psikhologa-s_detskoi...
[INFO] File exists. Skipping Psihology-Aykido...
[INFO] File exists. Skipping Robert-Chaldyny_Psyhologyya-vliyaniya_Kak-nauchitsya-ubezhdat-y-dobivatsya-uspeha...


In [2]:
 #\xa0

In [3]:
import fitz
from tqdm.auto import tqdm

def text_formatter(text : str) -> str:
    """Performs basic text formatting."""
    cleaned_text = text.replace("\n", " ").strip()
    return cleaned_text

def open_and_read_pdf(file_path: str) -> list[str]:
    doc = fitz.open(file_path)
    pages_and_texts = []
    for page_num, page in tqdm(enumerate(doc)):
        text = page.get_text("text")
        text = text_formatter(text=text)
        if "Goleman" in file_path:
            text = text.replace("\xa0", " ")
        pages_and_texts.append({"book": file_path[17:-4], "page_num": page_num, "page_char_count": len(text), "page_word_count": len(text.split()), "page_token_count": len(text)/4, "text": text})
    return  pages_and_texts

texts = []
for book in books:
    texts.extend(open_and_read_pdf(pdf_path + book + '.pdf'))
len(texts)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

1748

In [4]:
# Дополнительный препроцессинг текста

import re
from nltk import sent_tokenize  # Assuming NLTK is available or can be used; if not, implement custom sentence splitter

# Additional preprocessing steps:
# 1. Hyphen removal for word breaks: Merge words split by hyphens across lines, e.g., "паци- ентам" -> "пациентам". This uses regex to find patterns like "word- " followed by another word and joins them without the hyphen.
# 2. Multiple spaces normalization: Replace multiple whitespace characters with a single space to clean up extra gaps.
# 3. Remove special annotations: Strip out common footnote markers like "— Примеч. пер." using regex patterns for known formats.
# 4. Sentence tokenization: Split the cleaned text into individual sentences using NLTK's sent_tokenize, which handles Russian punctuation reasonably well. If NLTK isn't available, a custom regex-based splitter is provided as fallback.
# 5. Lowercase normalization (optional): Convert text to lowercase for consistency, but can be skipped if case sensitivity is needed.
# 6. Trim leading/trailing punctuation: Remove any leading or trailing non-alphabetic characters from sentences if they don't belong.

def advanced_text_formatter(text: str) -> str:
    """Enhanced text formatting beyond basic."""
    # Step 1: Remove hyphens from word breaks
    text = re.sub(r'(\w+)-\s+(\w+)', r'\1\2', text)
    
    # Step 2: Normalize multiple spaces
    text = re.sub(r'\s+', ' ', text)
    
    # Step 3: Remove specific annotations like footnotes
    text = re.sub(r'—\s*Примеч\.\s*пер\.\s*', '', text)  # Target "— Примеч. пер."
    text = re.sub(r'\(\s*\d+\s*\)\.', '', text)  # Remove numbered lists like "(1).", "(2)." etc.
    
    # Step 4: Optional lowercase
    # text = text.lower()  # Uncomment if needed
    
    return text.strip()

# Fallback sentence splitter if NLTK not available
def custom_sent_tokenize(text: str) -> list[str]:
    """Custom regex-based sentence tokenizer for Russian text."""
    sentence_end = re.compile(r'(?<!\w\.\w.)(?<![A-ZА-Я][a-zа-я]\.)(?<=\.|\\?|\\!|\…)\s')
    sentences = sentence_end.split(text)
    return [s.strip() for s in sentences if s.strip()]

# Apply advanced formatting to existing texts
for item in texts:
    item['text'] = advanced_text_formatter(item['text'])

# Now split into sentences
sentences = []
for item in texts:
    # Use NLTK if available
    try:
        page_sentences = sent_tokenize(item['text'], language='russian')
    except:
        page_sentences = custom_sent_tokenize(item['text'])
    
    for sent in page_sentences:
        # Step 6: Trim leading/trailing punctuation if unnecessary
        sent = re.sub(r'^[^\w]+', '', sent)  # Remove leading non-word chars
        sent = re.sub(r'[^\w]+$', '', sent)  # Remove trailing non-word chars
        
        sentences.append({
            "book": item['book'],
            "page_num": item['page_num'],
            "sentence": sent,
            "sent_char_count": len(sent),
            "sent_word_count": len(sent.split()),
            "sent_token_count": len(sent) / 4
        })

print(f"Total sentences extracted: {len(sentences)}")

# Optionally, save to file or further process
import json
with open('processed_sentences.json', 'w', encoding='utf-8') as f:
    json.dump(sentences, f, ensure_ascii=False, indent=4)

Total sentences extracted: 27035


In [5]:
texts[700]['text']

'Психика и медицина 281 Глава 11 Психика и медицина «Кто научил вас этому всему, доктор?» Ответ последовал незамедлительно: «Страдание». Альбер Камю . Чума Непонятная тупая боль в паху погнала меня в больницу. Осмотр не выявил ничего необычного, пока врач не увидел результаты анализа мочи: в ней были обнаружены следы крови. — Я хочу, чтобы вы легли в больницу и прошли коекакие исследования. Надо проверить работу почек, сделать цитологию… — сказал он деловым тоном. Не помню, что он говорил дальше. Мое сознание, казалось, застыло на слове «цитология». Рак. У меня осталось смутное воспоминание, что он объяснял, где и когда я должен пройти диагностику. Простейшие указания, но даже их пришлось просить несколько раз повторить. «Цитология…» — мой ум не желал расставаться с этим словом. Оно вызвало у меня такое чувство, будто меня сзади схватили за горло и грабят на пороге собственного дома.'

In [6]:
import pandas as pd

df = pd.DataFrame(texts)
df.head()


,book,page_num,page_char_count,page_word_count,page_token_count,text
0,Allana-Piza-YAzyk-telodvizhenij,0,0,0,0.00,
1,Allana-Piza-YAzyk-telodvizhenij,1,1378,182,344.50,Annotation Книга Аллана Пиза «Язык телодвижени...
2,Allana-Piza-YAzyk-telodvizhenij,2,1021,138,255.25,Различие Пространственных Зон у Горожан и Жите...
3,Allana-Piza-YAzyk-telodvizhenij,3,1052,142,263.00,"Жест Скрещивание Рук, Усиленное Сжатием Пальце..."
4,Allana-Piza-YAzyk-telodvizhenij,4,902,112,225.50,"Жесты, Используемые Мужчинами при Ухаживании Ж..."


In [7]:
df.describe().round(2)

,page_num,page_char_count,page_word_count,page_token_count
count,1748.00,1748.00,1748.00,1748.00
mean,187.05,1719.60,239.63,429.90
std,135.44,618.48,93.74,154.62
min,0.00,0.00,0.00,0.00
25%,74.00,1514.25,211.00,378.56
50%,161.00,1939.00,264.00,484.75
75%,275.00,2102.00,288.00,525.50
max,544.00,3273.00,1386.00,818.25


In [8]:
from spacy.lang.ru import Russian
nlp = Russian()

nlp.add_pipe("sentencizer")

doc = nlp('Это первое предложение. Это второе предложение! А это третье предложение?')
assert len(list(doc.sents)) == 3

list(doc.sents)

[Это первое предложение., Это второе предложение!, А это третье предложение?]

In [9]:
for item in tqdm(texts):
    item['sentences'] = list(nlp(item['text']).sents)

    item['sentences'] = [str(sent) for sent in item['sentences']]

    item['page_sentence_count_spacy'] = len(item['sentences'])

  0%|          | 0/1748 [00:00<?, ?it/s]

In [10]:
import random
random.sample(texts, k=1)

[{'book': 'Goleman-D.-Emotional-Intelligence.-Why-it-may-be-more-important-than-IQ',
  'page_num': 245,
  'page_char_count': 1960,
  'page_word_count': 283,
  'page_token_count': 490.0,
  'text': '246 Эмоциональный разум в действии или поздно обязательно произойдет взрыв. Но если все спорные темы регулярно улаживать, то напряженность снизится. Мужьям стоит также понять, что раздражение или недовольство вовсе не предполагают личных нападок: эмоции жен зачастую служат лишь показателем глубины их переживаний по тому или иному поводу. Мужчинам также нужно быть осторожными, чтобы не сорвать начавшееся обсуждение семейных проблем, слишком рано предложив практическое решение, — как правило, жене важнее ощущать, что муж выслушивает ее жалобы и проникается ее чувствами по поводу какогото вопроса (хотя ему совсем не обязательно соглашаться с ней). Она может расценить его попытку чтолибо посоветовать как способ дать ей понять, что ее чувства непоследовательны. Мужья, способные разделять общество 

In [11]:
data = pd.DataFrame(texts)
data.describe().round(2)


,page_num,page_char_count,page_word_count,page_token_count,page_sentence_count_spacy
count,1748.00,1748.00,1748.00,1748.00,1748.00
mean,187.05,1719.60,239.63,429.90,14.68
std,135.44,618.48,93.74,154.62,8.53
min,0.00,0.00,0.00,0.00,0.00
25%,74.00,1514.25,211.00,378.56,10.00
50%,161.00,1939.00,264.00,484.75,14.00
75%,275.00,2102.00,288.00,525.50,18.00
max,544.00,3273.00,1386.00,818.25,73.00


In [12]:
# chunking with spacy sentences (1 chunk = ?10? sentences). 

num_sentences_per_chunk = 10

def split_list(input_list: list[str], slice_size: int=num_sentences_per_chunk) -> list[list[str]]:
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]

test_list = list(range(25))
split_list(test_list)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24]]

In [13]:
import re


all_sentences = []
for item in tqdm(texts):
    for sent in item['sentences']:
        all_sentences.append({
            "sentence": str(sent),
            "book": item['book'],
            "page_num": item['page_num']
        })


sentence_chunks = split_list(all_sentences, slice_size=num_sentences_per_chunk)


pages_and_chunks = []
for chunk in tqdm(sentence_chunks):
    chunk_dict = {}
    chunk_dict['book'] = chunk[0]['book']
    chunk_dict['page_num'] = chunk[0]['page_num']
    
    # Join the sentences in the chunk
    joined_sentence_chunk = " ".join([item['sentence'] for item in chunk]).replace("  ", " ").strip()
    joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk)
    chunk_dict['sentence_text'] = joined_sentence_chunk
    
    # Recalculate stats for the new chunk
    chunk_dict['chunk_char_count'] = len(joined_sentence_chunk)
    chunk_dict['chunk_word_count'] = len(joined_sentence_chunk.split(" "))
    chunk_dict['chunk_token_count'] = len(joined_sentence_chunk) / 4
    
    pages_and_chunks.append(chunk_dict)

len(pages_and_chunks)

  0%|          | 0/1748 [00:00<?, ?it/s]

  0%|          | 0/2566 [00:00<?, ?it/s]

2566

In [14]:
random.sample(texts, k=1)

[{'book': 'Gurina-Koshenova-Rabota-psikhologa-s_detskoi',
  'page_num': 135,
  'page_char_count': 1496,
  'page_word_count': 186,
  'page_token_count': 374.0,
  'text': '136 3. Устойчивые искаженные суждения о причинах и/или последствиях травматических событий, которые заставляют индивидуума винить себя или других. 4. Устойчивые негативные эмоции (например, страх, гнев, вина, стыд). 5. Заметно сниженный интерес к участию в значимых для индивида мероприятиях либо отказ от участия в них. 6. Чувство отдаленности (отсутствие эмоциональной вовлеченности) в межличностных отношениях или отчуждения от других. 7. Устойчивая неспособность испытывать положительные эмоции (например, неспособность испытывать счастье, удовлетворение или любовь). E. Заметные изменения в возбуждении и реактивности, ассоциированные с травматическим событием (событиями) либо которые начинаются и/или усиливаются после самого события (событий) (два или более признака из нижеперечисленных): 1. Раздраженное поведение либо в

In [15]:
df = pd.DataFrame(texts)
df.describe().round(2)

,page_num,page_char_count,page_word_count,page_token_count,page_sentence_count_spacy
count,1748.00,1748.00,1748.00,1748.00,1748.00
mean,187.05,1719.60,239.63,429.90,14.68
std,135.44,618.48,93.74,154.62,8.53
min,0.00,0.00,0.00,0.00,0.00
25%,74.00,1514.25,211.00,378.56,10.00
50%,161.00,1939.00,264.00,484.75,14.00
75%,275.00,2102.00,288.00,525.50,18.00
max,544.00,3273.00,1386.00,818.25,73.00


In [16]:
import re
from tqdm.auto import tqdm

num_sentences_per_chunk = 10
def split_list(input_list: list, slice_size: int = num_sentences_per_chunk) -> list[list]:
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]

for item in tqdm(texts):
    if 'sentences' in item:
        item['sentence_chunks'] = split_list(item['sentences'])
        item['num_chunks'] = len(item['sentence_chunks'])

pages_and_chunks = []
for item in tqdm(texts):
    if 'sentence_chunks' in item:
        for sentence_chunk in item['sentence_chunks']:
            chunk_dict = {}
            chunk_dict['book'] = item['book']
            chunk_dict['page_num'] = item['page_num']
            joined_sentence_chunk = ' '.join(map(str, sentence_chunk)).replace("  ", " ").strip()
            joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk)
            chunk_dict['sentence_text'] = joined_sentence_chunk
            
            chunk_dict['chunk_char_count'] = len(joined_sentence_chunk)
            chunk_dict['chunk_word_count'] = len(joined_sentence_chunk.split(" "))
            chunk_dict['chunk_token_count'] = len(joined_sentence_chunk)/4

            pages_and_chunks.append(chunk_dict)

len(pages_and_chunks)

  0%|          | 0/1748 [00:00<?, ?it/s]

  0%|          | 0/1748 [00:00<?, ?it/s]

3427

In [17]:
random.sample(pages_and_chunks, k=1)

[{'book': 'Goleman-D.-Emotional-Intelligence.-Why-it-may-be-more-important-than-IQ',
  'page_num': 163,
  'sentence_text': '164 Природа эмоционального интеллекта настроения, вгоняет в апатию, а подчас и в депрессию. Оптимисты, напротив, говорят себе: «Я выбрал не тот подход» или: «Этот клиент просто был в плохом настроении». Считая причиной неудачи в конкретной ситуации не себя, а нечто другое, они могут изменить подход к продажам при разговоре со следующим клиентом. И если ментальная установка пессимиста ведет к отчаянию, то настрой оптимиста внушает ему надежду. Одним из источников позитивного или негативного мировоззрения вполне может быть врожденный характер; некоторые люди по природе склонны к тому или иному образу действий. Но, как мы узнáем из главы 14, жизненный опыт может закалить характер. Оптимизму и надежде (как и беспомощности и отчаянию) можно научиться. В основе того и другого лежит качество, которое психологи называют самоэффективностью — верой в то, что ты обладаешь со

In [18]:
df = pd.DataFrame(texts)
df.describe().round(2)

,page_num,page_char_count,page_word_count,page_token_count,page_sentence_count_spacy,num_chunks
count,1748.00,1748.00,1748.00,1748.00,1748.00,1748.00
mean,187.05,1719.60,239.63,429.90,14.68,1.96
std,135.44,618.48,93.74,154.62,8.53,0.88
min,0.00,0.00,0.00,0.00,0.00,0.00
25%,74.00,1514.25,211.00,378.56,10.00,1.00
50%,161.00,1939.00,264.00,484.75,14.00,2.00
75%,275.00,2102.00,288.00,525.50,18.00,2.00
max,544.00,3273.00,1386.00,818.25,73.00,8.00


In [19]:
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_num,chunk_char_count,chunk_word_count,chunk_token_count
count,3427.00,3427.00,3427.00,3427.00
mean,191.52,862.12,118.67,215.53
std,142.89,505.03,67.04,126.26
min,0.00,2.00,1.00,0.50
25%,70.50,453.00,63.00,113.25
50%,159.00,806.00,114.00,201.50
75%,286.00,1260.00,173.00,315.00
max,543.00,2214.00,293.00,553.50


In [20]:
min_token_length = 30
for row in df[df['chunk_token_count'] <= min_token_length].sample(5).iterrows():

    print(f'Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_text"]}')

Chunk token count: 1.0 | Text: ред.
Chunk token count: 1.0 | Text: ред.
Chunk token count: 8.75 | Text: Однако позднее, когда он уже не 132
Chunk token count: 21.25 | Text: При первой появившейся возможности она получит свои конфеты обратно с дополнением, 84
Chunk token count: 1.0 | Text: ред.


In [21]:
pages_and_chunks_over_min_token_len = df[df['chunk_token_count'] > min_token_length].to_dict(orient='records')
pages_and_chunks_over_min_token_len[20:25]

[{'book': 'Allana-Piza-YAzyk-telodvizhenij',
  'page_num': 17,
  'sentence_text': 'Конгруэнтность — Совпадение Слов и Жестов Если бы вы были собеседником человека, показанного на рис. 4, и попросили его выразить свое мнение относительно того, что вы только что сказали, на что он бы ответил, что с вами не согласен, то его невербальные сигналы были бы конгруэнтными, т.е. соответ-ствовали бы его словесным высказываниям. Если же он скажет, что ему очень нравится все, что вы говорите, он будет лгать, потому что его слова и жесты будут не конгруэнтными. Исследования доказывают, что невербальные сигналы несут в 5 раз больше информации, чем вербальные, и в случае, если сигналы не — конгруэнтны, люди полагаются на невербальную информацию, предпочитая ее словесной. Часто можно наблюдать, как какой-нибудь политик стоит на трибуне, крепко скрестив руки на груди ( защитная поза ) с опущенным подбородком ( критическая или враждебная поза), и говорит аудитории о том, как восприимчиво и дружелюбно он 

In [22]:
random.sample(pages_and_chunks_over_min_token_len, k=1)

[{'book': 'Gurina-Koshenova-Rabota-psikhologa-s_detskoi',
  'page_num': 26,
  'sentence_text': 'Несмотря на то что ребенок защищен от психогенной смерти способностью засыпать в состоянии тотального несчастья и истощения, повторение таких эпизодов приводит к неудачам развития, истощению или анаклитической депрессии. Взрослая травма – переживание неизбежной опасности. Травма относится к переполняющему парализующему психическому состоянию, включающему в себя утрату функций эго, остановку функций саморефлексии, паралич всех когнитивных функций и функций самосохранения, тревожность или кататонию вплоть до психогенной смерти. Взрослую травму в психоанализе можно назвать термином «катастрофическая», иногда этот термин употребляется и в отношении детей («детская катастрофическая травма»), в этом случае имеется в виду, что событие, оказавшее травмирующее воздействие, пришлось на период детства. Можно выделить общие особенности взрослой и инфантильной травмы: 1) чувство беспомощности;',
  'chunk

## Эмбендинг

In [23]:
import requests

model_id = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2" #hugging face model
hf_token = "get your token in http://hf.co/settings/tokens"

from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name_or_path=model_id, device='cuda')

sentences = ["Своевременное управление своими эмоциями помогает человеку лучше адаптироваться в обществе и достигать успехов в различных сферах жизни.", 
             "Эмоциональный интеллект включает в себя способность распознавать, понимать и управлять своими эмоциями, а также эмоциями других людей.",
             "Мне нравятся лошади"]

embeddings = embedding_model.encode(sentences)
embeddings_dictionary = dict(zip(sentences, embeddings))

for sentence, embedding in embeddings_dictionary.items():
    print(f'Sentence: {sentence}\\nEmbedding: {embedding[:5]}\\n')

Sentence: Своевременное управление своими эмоциями помогает человеку лучше адаптироваться в обществе и достигать успехов в различных сферах жизни.\nEmbedding: [ 0.09367368  0.06438737  0.15397337  0.27672583 -0.2889965 ]\n
Sentence: Эмоциональный интеллект включает в себя способность распознавать, понимать и управлять своими эмоциями, а также эмоциями других людей.\nEmbedding: [-0.18342017 -0.00970363  0.11679671  0.56001985 -0.31229788]\n
Sentence: Мне нравятся лошади\nEmbedding: [ 0.09567391  0.13053113  0.00370117  0.2758525  -0.23311627]\n


In [24]:
embeddings[0].shape

(384,)

%%time

embedding_model.to('cpu')

for item in tqdm(pages_and_chunks_over_min_token_len):
    item['embedding'] = embedding_model.encode(item['sentence_text'])

In [25]:
%%time

embedding_model.to('cuda')

for item in tqdm(pages_and_chunks_over_min_token_len):
    item['embedding'] = embedding_model.encode(item['sentence_text'])

  0%|          | 0/3245 [00:00<?, ?it/s]

CPU times: total: 37.5 s
Wall time: 37.3 s


In [26]:
%%time
embedding_model.to('cuda')

text_chunks = [item["sentence_text"] for item in pages_and_chunks_over_min_token_len]
text_chunks[419]

CPU times: total: 0 ns
Wall time: 3 ms


'Глава 5. Игры 55 B. Наконец, игра третьей степени — это игра безудержная; доведённая до конца, она завершается в больнице, в суде или морге. Игры можно классифицировать также и по другим специфическим признакам, указанным при анализе ВИТ: по целям, ролям, наиболее очевидным преимуществам. Для систематической научной классификации наиболее подходящей может оказаться, вероятно, экзистенциальная точка зрения; но поскольку этот фактор ещё недостаточно изучен, такую классификацию придётся отложить на будущее. За неимением её, в настоящее время самой удобной классификацией представляется социологическая, которой мы и будем придерживаться в следующей части. Примечания Необходимо отметить заслуги Стивена Поттера с его чутким, пронизанным юмором анализом манёвров или “проделок” в повседневных жизненных ситуациях [2], и Дж. Г. Мида, пионера в исследовании общественной роли игр [3]. Игры, ведущие к психиатрическим расстройствам, систематически изучались на Сан-Францисских семинарах по социальной

In [27]:
len(text_chunks)

3245

In [28]:
%%time

text_chunk_embeddings = embedding_model.encode(text_chunks, batch_size=32, convert_to_tensor=True)

text_chunk_embeddings

CPU times: total: 8.64 s
Wall time: 2.94 s


tensor([[-0.0269, -0.2128, -0.1893,  ...,  0.1641, -0.1914,  0.0047],
        [ 0.0649,  0.0589,  0.1032,  ...,  0.0837, -0.1090,  0.0020],
        [-0.0734,  0.0245,  0.1093,  ...,  0.0689, -0.0012,  0.3155],
        ...,
        [-0.0400, -0.0541, -0.0252,  ...,  0.0221,  0.0211,  0.0897],
        [-0.0159, -0.0377, -0.2868,  ...,  0.1625, -0.0704,  0.1459],
        [-0.0337,  0.0736, -0.1271,  ...,  0.1303, -0.0517,  0.1671]],
       device='cuda:0')

## Сохранение эмбеддингов в файл

In [29]:
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)
embeddings_df_save_path = 'text_chunks_and_embeddings_df.csv'
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)

In [30]:
embeddings_df_save_path = 'text_chunks_and_embeddings_df.csv'
text_chunks_and_embeddings_df = pd.read_csv(embeddings_df_save_path)
text_chunks_and_embeddings_df.head()

,book,page_num,sentence_text,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,Allana-Piza-YAzyk-telodvizhenij,1,Annotation Книга Аллана Пиза «Язык телодвижени...,1377,183,344.25,[-0.02687061 -0.212832 -0.18931839 -0.023208...
1,Allana-Piza-YAzyk-telodvizhenij,2,Различие Пространственных Зон у Горожан и Жите...,1021,138,255.25,[ 0.06494357 0.05886629 0.10323059 -0.055009...
2,Allana-Piza-YAzyk-telodvizhenij,3,"Жест Скрещивание Рук, Усиленное Сжатием Пальце...",1052,142,263.00,[-7.34471381e-02 2.44905725e-02 1.09274082e-...
3,Allana-Piza-YAzyk-telodvizhenij,4,"Жесты, Используемые Мужчинами при Ухаживании Ж...",893,112,223.25,[-0.01165801 0.00611222 0.04863156 0.022569...
4,Allana-Piza-YAzyk-telodvizhenij,5,Как Ступени Ног Выражают Заинтересованность Ра...,764,103,191.00,[ 0.1991961 -0.00937766 -0.03220402 -0.164424...


Если бы у меня была действительно большая база данных эмбэндингов (100K - 1M) то тогда бы использовала векторную базу данных qdrant

# Поиск и ответы

In [31]:
import random
import torch
import numpy as np
import pandas as pd
device = "cuda"



text_chunks_and_embeddings_df = pd.read_csv("text_chunks_and_embeddings_df.csv")


text_chunks_and_embeddings_df['embedding'] = text_chunks_and_embeddings_df['embedding'].apply(lambda x: np.fromstring(x.strip("[]"), sep=' '))

embeddings = torch.tensor(np.stack(text_chunks_and_embeddings_df['embedding'].tolist(), axis=0), dtype=torch.float32).to(device)

pages_and_chunks = text_chunks_and_embeddings_df.to_dict(orient='records')

text_chunks_and_embeddings_df

,book,page_num,sentence_text,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,Allana-Piza-YAzyk-telodvizhenij,1,Annotation Книга Аллана Пиза «Язык телодвижени...,1377,183,344.25,"[-0.02687061, -0.212832, -0.18931839, -0.02320..."
1,Allana-Piza-YAzyk-telodvizhenij,2,Различие Пространственных Зон у Горожан и Жите...,1021,138,255.25,"[0.06494357, 0.05886629, 0.10323059, -0.055009..."
2,Allana-Piza-YAzyk-telodvizhenij,3,"Жест Скрещивание Рук, Усиленное Сжатием Пальце...",1052,142,263.00,"[-0.0734471381, 0.0244905725, 0.109274082, -0...."
3,Allana-Piza-YAzyk-telodvizhenij,4,"Жесты, Используемые Мужчинами при Ухаживании Ж...",893,112,223.25,"[-0.01165801, 0.00611222, 0.04863156, 0.022569..."
4,Allana-Piza-YAzyk-telodvizhenij,5,Как Ступени Ног Выражают Заинтересованность Ра...,764,103,191.00,"[0.1991961, -0.00937766, -0.03220402, -0.16442..."
...,...,...,...,...,...,...,...
3240,Robert-Chaldyny_Psyhologyya-vliyaniya_Kak-nauc...,403,"Journal of Personality and Social Psychology, ...",742,108,185.50,"[-0.2240675, 0.03327263, 0.05968623, 0.2548973..."
3241,Robert-Chaldyny_Psyhologyya-vliyaniya_Kak-nauc...,404,"University, Institute for Research in the Beha...",789,111,197.25,"[-0.00610837713, 0.0936156958, 0.0434962139, 0..."
3242,Robert-Chaldyny_Psyhologyya-vliyaniya_Kak-nauc...,405,убийства в Вирджинии и Северном Иллинойсе). 2....,955,124,238.75,"[-0.0400359742, -0.0540543161, -0.0251563713, ..."
3243,Robert-Chaldyny_Psyhologyya-vliyaniya_Kak-nauc...,405,"Удвоившееся количество отчетов читателей, их р...",631,82,157.75,"[-0.01591558, -0.03768359, -0.2867751, 0.10031..."


In [32]:
random.sample(list(text_chunks_and_embeddings_df['sentence_text']), k=1)

['268 16. Дольто Ф. На стороне ребенка = La cause des enfants / пер. с фр. Е. В. Баевской, О. В. Давтян; предисл. М. М. Безруких. – Екатеринбург: Рама Паблишинг, 2010. – 717 с. – (Авторитетные детские психологи) 17. Есина О. Б. Посттравматическое стрессовое расстройство у детей: психосоциальные факторы формирования, клинические характеристики и проблемы диагностики: дис. … канд. мед. наук. – СПб.: Санкт-Петербургский науч.- исслед.']

In [33]:
embeddings.shape

torch.Size([3245, 384])

In [34]:
from sentence_transformers import SentenceTransformer, util
embedding_model = SentenceTransformer(model_name_or_path=model_id, device=device)

In [35]:
from time import perf_counter as timer

query = 'Здоровые отношения между женщиной и мужчиной'
print(f'Query: {query}')

query_embedding = embedding_model.encode(query, convert_to_tensor=True).to(device)

print(query_embedding.shape)


cosine_scores = util.cos_sim(query_embedding, embeddings)[0]

start_time = timer()
dot_scores = util.dot_score(query_embedding, embeddings)[0]
end_time = timer()

print(f'[INFO] Time taken to compute dot product scores: {end_time - start_time:.4f} seconds \\n len = {len(embeddings)}')

top_results = torch.topk(cosine_scores, k=5)
top_results

Query: Здоровые отношения между женщиной и мужчиной
torch.Size([384])
[INFO] Time taken to compute dot product scores: 0.0001 seconds \n len = 3245


torch.return_types.topk(
values=tensor([0.6334, 0.6161, 0.5887, 0.5878, 0.5758], device='cuda:0'),
indices=tensor([1073, 1208,  996, 2521, 1075], device='cuda:0'))

In [36]:
for i in top_results.indices.tolist():
    print(text_chunks_and_embeddings_df['sentence_text'][i])

Получается, что женщины вступают в брак готовыми к роли эмоционального управителя, тогда как мужчины в гораздо меньшей степени понимают важность стратегии для сохранения отношений. По результатам опроса 264 пар, самой важной составляющей удовлетворенности взаимоотношениями для женщин — но не для мужчин! — было ощущение, что между супругами хорошо налажено общение9. Тед Хастон, психолог из Техасского университета, изучавший супружеские пары, замечает: «Что касается жен, то для них близость подразумевает обсуждение всех дел, особенно разговоры о самих отношениях. Мужчины же не понимают, чего жены от них хотят. Они
Психика и медицина 307 лет, чем медицинские показатели (нестабильное кровяное давление, повышение концентрации триглицеридов в крови или уровня холестерина). Однако среди мужчин, имевших, по их словам, вполне надежную сеть близких контактов, включающую, например, жену, детей, друзей, не обнаружено никакой взаимосвязи между высоким уровнем стресса и коэффициентом смертности. Сам

In [37]:
larger_embeddings = torch.randn(1000 * embeddings.shape[0], embeddings.shape[1]).to(device)
print(f"Larger embeddings shape: {larger_embeddings.shape}")

start_time = timer()
dot_scores = util.dot_score(query_embedding, larger_embeddings)[0]
end_time = timer()

print(f'[INFO] Time taken to compute dot product scores with larger embeddings: {end_time - start_time:.4f} seconds \\n len = {len(larger_embeddings)}')

Larger embeddings shape: torch.Size([3245000, 384])
[INFO] Time taken to compute dot product scores with larger embeddings: 0.0031 seconds \n len = 3245000


In [38]:
import textwrap

def print_wrapped(text: str, width: int=80):
    wrapped_text = textwrap.fill(text, width=width)
    print(wrapped_text)

In [39]:
print(f"Query: {query}\\n")
print("Results:")

for score, idx in zip(top_results[0], top_results[1]):
    print_wrapped(f"Score: {score:.4f} | Text: {pages_and_chunks[idx]['sentence_text']}\\n")

Query: Здоровые отношения между женщиной и мужчиной\n
Results:
Score: 0.6334 | Text: Получается, что женщины вступают в брак готовыми к роли
эмоционального управителя, тогда как мужчины в гораздо меньшей степени понимают
важность стратегии для сохранения отношений. По результатам опроса 264 пар,
самой важной составляющей удовлетворенности взаимоотношениями для женщин — но не
для мужчин! — было ощущение, что между супругами хорошо налажено общение9. Тед
Хастон, психолог из Техасского университета, изучавший супружеские пары,
замечает: «Что касается жен, то для них близость подразумевает обсуждение всех
дел, особенно разговоры о самих отношениях. Мужчины же не понимают, чего жены от
них хотят. Они\n
Score: 0.6161 | Text: Психика и медицина 307 лет, чем медицинские показатели
(нестабильное кровяное давление, повышение концентрации триглицеридов в крови
или уровня холестерина). Однако среди мужчин, имевших, по их словам, вполне
надежную сеть близких контактов, включающую, например, жену, д

## Напишем пайплайн нашего семантического поиска

In [40]:
def retrive_relevant_resources(query: str, embeddings: torch.tensor, model: SentenceTransformer, n_resources_to_return: int=5, print_time: bool=True):
    query_embedding = model.encode(query, convert_to_tensor=True)

    start_time = timer()
    dot_scores = util.dot_score(query_embedding, embeddings)[0].to(embeddings.device)
    end_time = timer()
    if print_time:
        print(f"[INFO] Time taken to get scores on {len(embeddings)} embeddings: {end_time - start_time:.5f} seconds")
    
    scores, indices = torch.topk(dot_scores, k=n_resources_to_return)
    return scores, indices


def print_top_results_and_scores(query: str, embeddings: torch.tensor, pages_and_chunks: list[dict] = pages_and_chunks, n_resources_to_return: int=5, print_time: bool=True):
    scores, indices = retrive_relevant_resources(query=query, embeddings=embeddings, model=embedding_model, n_resources_to_return=n_resources_to_return)
    
    for score, idx in zip(scores, indices):
        print_wrapped(f"Score: {score:.4f} | Text: {pages_and_chunks[idx]['sentence_text']}\\n")


In [41]:
query="Как выявить ложь"
embeddings = embeddings.to(query_embedding.device)
retrive_relevant_resources(query=query, embeddings=embeddings, model=embedding_model, n_resources_to_return=5)
print_top_results_and_scores(query=query, embeddings=embeddings, n_resources_to_return=5)

[INFO] Time taken to get scores on 3245 embeddings: 0.00009 seconds
[INFO] Time taken to get scores on 3245 embeddings: 0.00007 seconds
Score: 9.8382 | Text: Как Сказать Неправду, не Раскрыв Себя Проблема с ложью
заключается в том, что наше подсознание работает автоматически и независимо от
нас, поэтому наш язык телодвижений выдает нас с головой. Вот почему сразу
заметно, когда лгут люди, редко говорящие неправду, независимо от того,
насколько убедительно они это преподносят. В тот самый момент, когда они
начинают лгать, их тело начинает давать совершенно противоположные сигналы, что
дает вам ощущение, что вам лгут. Во время обмана наше подсознание выбрасывает
пучок нервной энергии, которая проявляется в жестах, противоречащих тому, что
сказал человек. Некоторые ? 1юди, чьи профессии непосредственно связаны с
обманом в разных формах, такие как политические деятели, адвокаты, актеры и
телекомментаторы, до такой степени выдрессировали свои телодвижения, что у них
трудно заметить, что они

## LLM локальная генерация

In [42]:
import torch
gpu_memory_bytes = torch.cuda.get_device_properties(0).total_memory
gpu_memory_gb = gpu_memory_bytes / (1024 ** 3)
print(f"GPU Memory: {gpu_memory_gb:.2f} GB")


GPU Memory: 8.00 GB


In [43]:
# Note: the following is Gemma focused, however, there are more and more LLMs of the 2B and 7B size appearing for local use.
if gpu_memory_gb < 5.1:
    print(f"Your available GPU memory is {gpu_memory_gb}GB, you may not have enough memory to run a Gemma LLM locally without quantization.")
elif gpu_memory_gb < 8.1:
    print(f"GPU memory: {gpu_memory_gb} | Recommended model: Gemma 2B in 4-bit precision.")
    use_quantization_config = True 
    model_id = "google/gemma-2b-it"
elif gpu_memory_gb < 19.0:
    print(f"GPU memory: {gpu_memory_gb} | Recommended model: Gemma 2B in float16 or Gemma 7B in 4-bit precision.")
    use_quantization_config = False 
    model_id = "google/gemma-2b-it"
elif gpu_memory_gb > 19.0:
    print(f"GPU memory: {gpu_memory_gb} | Recommend model: Gemma 7B in 4-bit or float16 precision.")
    use_quantization_config = False 
    model_id = "google/gemma-7b-it"

print(f"use_quantization_config set to: {use_quantization_config}")
print(f"model_id set to: {model_id}")


GPU memory: 7.99969482421875 | Recommended model: Gemma 2B in 4-bit precision.
use_quantization_config set to: True
model_id set to: google/gemma-2b-it


## Загрузка локального LLM

Попробуем так. Но если результаты будут неудовлетворительные, то лучше перейти в google colab

In [44]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers.utils import is_flash_attn_2_available
import time
import subprocess

# Логирование CUDA и VRAM
print("Проверка CUDA:")
print(f"torch.cuda.is_available(): {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Количество GPU: {torch.cuda.device_count()}")
    print(f"Имя GPU: {torch.cuda.get_device_name(0)}")
    torch.cuda.empty_cache()  # Очистка кэша GPU
    total_vram = torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)
    used_vram = torch.cuda.memory_allocated(0) / (1024 ** 3)
    free_vram = total_vram - used_vram
    print(f"Всего VRAM: {total_vram:.2f} GiB, Занято: {used_vram:.2f} GiB, Свободно: {free_vram:.2f} GiB")
    if free_vram < 5:
        raise RuntimeError("Недостаточно VRAM! Освободите память и перезапустите.")
    try:
        print("nvidia-smi:")
        print(subprocess.check_output("nvidia-smi").decode("utf-8"))
    except:
        pass

# Конфигурация квантизации
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

# Attention
if is_flash_attn_2_available() and torch.cuda.get_device_capability(0)[0] >= 8:
    attn_implementation = "flash_attention_2"
else:
    attn_implementation = "eager"

model_id = "Qwen/Qwen3-8B"

# Токенизатор
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

# Загрузка модели с принудительным GPU
print("Загрузка модели...")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    attn_implementation=attn_implementation,
    torch_dtype="auto",
    device_map="cuda",  # Принудительно на GPU (изменено с "auto")
    trust_remote_code=True,
    llm_int8_enable_fp32_cpu_offload=True  # На всякий случай
)

# Логи устройства
print(f"Устройство модели: {next(model.parameters()).device}")
if 'cuda' not in str(next(model.parameters()).device):
    print("Внимание: Модель на CPU! Проверьте VRAM и bitsandbytes.")
print(torch.cuda.memory_summary(device=0, abbreviated=True))

# Генерация
messages = [{"role": "user", "content": "Who are you?"}]
text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer([text], return_tensors="pt").to(model.device)

print(f"Устройство инпутов: {inputs['input_ids'].device}")

start_time = time.time()
outputs = model.generate(**inputs, max_new_tokens=512, temperature=0.6, top_p=0.95, top_k=20)
end_time = time.time()

print(f"Время генерации: {end_time - start_time:.2f} секунд")
print("nvidia-smi после генерации:")
try:
    print(subprocess.check_output("nvidia-smi").decode("utf-8"))
except:
    pass

print(tokenizer.decode(outputs[0]))

Проверка CUDA:
torch.cuda.is_available(): True
Количество GPU: 1
Имя GPU: NVIDIA GeForce RTX 2080 SUPER
Всего VRAM: 8.00 GiB, Занято: 5.11 GiB, Свободно: 2.89 GiB


RuntimeError: Недостаточно VRAM! Освободите память и перезапустите.